In [ ]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 20000

In [ ]:
data_path = '/content/sample_data/fra.txt'

In [ ]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')

    # We use 'tab' as the 'start sequence' character
    # for the targets, and '\n' as the 'end sequence' character.

    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
len(output_characters)

101

In [ ]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

In [ ]:
print("Number of Samples:", len(input_texts))
print('Number of unique input Tokens:' , num_encoder_tokens)
print('Number of unique output Tokens:' , num_decoder_tokens)
print('Max sequence length for inputs:' , max_encoder_seq_length)
print('Max sequence length for outputs:' , max_decoder_seq_length)

Number of Samples: 20000
Number of unique input Tokens: 74
Number of unique output Tokens: 101
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [ ]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)])

output_token_index = dict(
    [(char,i) for i,char in enumerate(output_characters)])

In [ ]:
input_token_index , output_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '/': 10,
  '0': 11,
  '1': 12,
  '2': 13,
  '3': 14,
  '4': 15,
  '5': 16,
  '6': 17,
  '7': 18,
  '8': 19,
  '9': 20,
  ':': 21,
  '?': 22,
  'A': 23,
  'B': 24,
  'C': 25,
  'D': 26,
  'E': 27,
  'F': 28,
  'G': 29,
  'H': 30,
  'I': 31,
  'J': 32,
  'K': 33,
  'L': 34,
  'M': 35,
  'N': 36,
  'O': 37,
  'P': 38,
  'Q': 39,
  'R': 40,
  'S': 41,
  'T': 42,
  'U': 43,
  'V': 44,
  'W': 45,
  'Y': 46,
  'a': 47,
  'b': 48,
  'c': 49,
  'd': 50,
  'e': 51,
  'f': 52,
  'g': 53,
  'h': 54,
  'i': 55,
  'j': 56,
  'k': 57,
  'l': 58,
  'm': 59,
  'n': 60,
  'o': 61,
  'p': 62,
  'q': 63,
  'r': 64,
  's': 65,
  't': 66,
  'u': 67,
  'v': 68,
  'w': 69,
  'x': 70,
  'y': 71,
  'z': 72,
  'é': 73},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '4': 17,
  '5'

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length , num_decoder_tokens), dtype = 'float32')
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens) , dtype = 'float32')

In [ ]:
for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.

    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,output_token_index[char]] = 1.
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i,t-1,output_token_index[char]] = 1.
    decoder_input_data[i,t+1:, output_token_index[' ']] = 1.
    decoder_output_data[i , t: , output_token_index[' ']] = 1.

In [ ]:
encoder_input_data[0].shape

(17, 74)

In [ ]:
# define an input sequence and process it:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs,state_h,state_c = encoder(encoder_inputs)
# we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h,state_c]

In [ ]:
# set up the decoder , using encoder_states as initial states:
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state = True)
decoder_outputs, _ ,_ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Your existing code here...

# Compile and fit the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# Make predictions
predictions = model.predict([encoder_input_data, decoder_input_data])
predicted_labels = np.argmax(predictions, axis=-1)
true_labels = np.argmax(decoder_output_data, axis=-1)

# Flatten the arrays for confusion matrix computation
predicted_labels_flat = predicted_labels.flatten()
true_labels_flat = true_labels.flatten()

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels_flat, predicted_labels_flat)
print('Confusion Matrix:')
print(conf_matrix)

# Compute accuracy, precision, recall, and F1-score
accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)
precision = precision_score(true_labels_flat, predicted_labels_flat, average='weighted')
recall = recall_score(true_labels_flat, predicted_labels_flat, average='weighted')
f1 = f1_score(true_labels_flat, predicted_labels_flat, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


Epoch 1/100
250/250 [==============================] - 8s 17ms/step - loss: 0.1468 - accuracy: 0.9541 - val_loss: 0.5409 - val_accuracy: 0.8737
Epoch 2/100
250/250 [==============================] - 3s 12ms/step - loss: 0.1448 - accuracy: 0.9551 - val_loss: 0.5434 - val_accuracy: 0.8733
Epoch 3/100
250/250 [==============================] - 4s 15ms/step - loss: 0.1434 - accuracy: 0.9554 - val_loss: 0.5449 - val_accuracy: 0.8742
Epoch 4/100
250/250 [==============================] - 3s 13ms/step - loss: 0.1422 - accuracy: 0.9558 - val_loss: 0.5429 - val_accuracy: 0.8743
Epoch 5/100
250/250 [==============================] - 3s 12ms/step - loss: 0.1409 - accuracy: 0.9560 - val_loss: 0.5462 - val_accuracy: 0.8737
Epoch 6/100
250/250 [==============================] - 3s 12ms/step - loss: 0.1394 - accuracy: 0.9567 - val_loss: 0.5495 - val_accuracy: 0.8734
Epoch 7/100
250/250 [==============================] - 4s 15ms/step - loss: 0.1381 - accuracy: 0.9569 - val_loss: 0.5520 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9576915254237288
Precision: 0.9575551295495514
Recall: 0.9576915254237288
F1-score: 0.9574536207044317


In [ ]:
# define sampling models:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_input_states = [decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs, initial_state=decoder_input_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs]+decoder_input_states,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_output_char_index = dict(
    (i,char) for char,i in output_token_index.items())

def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    output_seq = np.zeros((1,1,num_decoder_tokens))
    output_seq[0,0,output_token_index['\t']] = 1

    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [output_seq]+ states_value)

        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char

        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True

        #update the target sequence (of length 1):
        output_seq = np.zeros((1,1,num_decoder_tokens))
        output_seq[0,0,sampled_token_index] = 1

        states_value = [h,c]
    return decoded_sentences

for seq_index in range(20):
    # take one sequence for trying out decoding:
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:' , decoded_sentences)

1/1 [==============================] - 0s 20ms/step
Input sentence: Go.
Decoded sentence: Va aie-toi !

1/1 [==============================] - 0s 25ms/step
Input sentence: Go.
Decoded sentence: Va aie-toi !

1/1 [==============================] - 0s 25ms/step
Input sentence: Go.
Decoded sentence: Va aie-toi !

1/1 [==============================] - 0s 23ms/step
Input sentence: Go.
Decoded sentence: Va aie-toi !

1/1 [==============================] - 0s 34ms/step
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 40ms/step
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 24ms/step
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 21ms/step
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 23ms/step
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 20ms/step
Input sentence: Run!
Decoded sentenc